In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator

In [2]:
# df.to_csv('btcusd.csv', index=False)

In [5]:
def apply_indicators(df):
    
    # Calculate the moving averages
    df['MA_daily'] = df['close'].rolling(window=50).mean()
    df['MA_weekly'] = df['close'].rolling(window=200).mean()

    # Calculate the Relative Strength Index (RSI)
    rsi_indicator = RSIIndicator(df['close'])
    df['RSI'] = rsi_indicator.rsi()

    # Calculate Bollinger Bands
    bollinger = BollingerBands(df['close'])
    df['BB_High'] = bollinger.bollinger_hband()
    df['BB_Low'] = bollinger.bollinger_lband()

    # Calculate MACD
    macd_indicator = MACD(df['close'])
    df['MACD'] = macd_indicator.macd()

    # Remove nas
    data = df.dropna()
    
    return data

# Getting data from the metatrader server

In [6]:
def get_data(symbol):

    # Getting data on the 15 minutes timeframe
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    # set the symbol and timefram
    timeframe = mt5.TIMEFRAME_M15   # for one-minute bars

    #dates for retrieving the history
    date_from = datetime(2016, 12, 15)
    today = datetime.today()

    # get the history
    history = mt5.copy_rates_range(symbol, timeframe, date_from, today)

    if history is not None and len(history) > 0:
        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(history).drop(['spread','real_volume'], axis =1)

        # convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    else:
        print("No data for the requested period")

    # terminate the connection to the MetaTrader 5 terminal
    mt5.shutdown()
    
    #adding indicators to retrieved data
    data = apply_indicators(rates_frame)
    
    
    return data

In [8]:
some =get_data('BTCUSD')
some

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,BB_Low,MACD
199,2017-04-06 18:45:00,1169.556,1170.112,1165.000,1165.000,129,1147.64562,1111.294980,64.165505,1165.367550,1136.566250,4.101407
200,2017-04-06 19:00:00,1165.000,1169.998,1165.000,1165.068,311,1148.26652,1111.914210,64.218324,1167.362602,1136.077098,4.499338
201,2017-04-06 19:15:00,1165.069,1169.995,1165.067,1168.000,226,1148.99152,1112.548045,66.510441,1169.668816,1136.143484,4.993725
202,2017-04-06 19:30:00,1168.000,1168.000,1161.173,1167.004,258,1149.62892,1113.173055,64.987490,1171.292217,1136.920283,5.244703
203,2017-04-06 19:45:00,1167.004,1169.988,1163.011,1166.116,245,1150.13170,1113.793635,63.589450,1172.573559,1137.750541,5.310733
...,...,...,...,...,...,...,...,...,...,...,...,...
211486,2023-06-13 10:30:00,26136.388,26159.638,26123.241,26124.738,374,25995.40872,26275.414200,56.604014,26168.817335,26038.560665,34.068034
211487,2023-06-13 10:45:00,26124.738,26153.338,26124.738,26143.238,291,26005.70072,26273.158200,58.882339,26172.331819,26042.906181,33.612305
211488,2023-06-13 11:00:00,26143.238,26197.738,26127.838,26170.238,677,26016.55872,26270.885700,62.016611,26181.155672,26042.552328,35.026052
211489,2023-06-13 11:15:00,26170.238,26186.738,26108.838,26144.738,633,26026.58072,26268.489700,57.554421,26184.585875,26043.512125,33.700343
